In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/4.vehicle'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

import json

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/3.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input = df_input[df_input['Json_Compatibility'] != json.dumps({})].reset_index(drop=True)

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        (crawler_index, crawler_series) = crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            list_dict_correct_temp, dict_ = [], json.loads(crawler_series['Json_Compatibility'])
            data = {'scopedContext': {'catalogDetails': {'categoryId': dict_['category_id'],
                                                         'itemId': dict_['item_id'],
                                                         'marketplaceId': dict_['market_place_id'],
                                                         'sellerName': dict_['seller_name']}}}
            for page in range(dict_['page']):

                count_retry = 0
                while True:
                    count_retry += 1

                    try:
                        url_check = crawler_series['Url']

                        resp = requests.post(f'''{dict_['api']}?module_groups=PART_FINDER&referrer=VIEWITEM&offset={page * 20}&module=COMPATIBILITY_TABLE''',
                                             data=json.dumps(data),
                                             headers=get_header(**{'Content-Type': 'application/json; charset=utf-8'}),
                                             proxies=get_proxy(),
                                             timeout=(5, 15))

                        if resp.status_code == 200:
                            list_header, list_row = [header['textSpans'][0]['text'].strip() for header in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']], [[row['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in row else row['textSpans'][0]['text'].strip() for row in dict_row['cells']] for dict_row in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']]

                            if list_header and list_row:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_dict_correct_temp_temp = [{'Json_Compatibility': crawler_series['Json_Compatibility'],
                                                'Page': page + 1,
                                                'Row': i + 1} for i in range(len(list_row))]

                for i, row in enumerate(list_row):
                    for j, header in enumerate(list_header):
                        list_dict_correct_temp_temp[i][header] = row[j]

                list_dict_correct_temp.extend(list_dict_correct_temp_temp)

                # = = = = = = = = = = = = = = =

                if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 30 == 0 and count_remain != 0:
                    clear_output()
                print(f'''[第{page + 1}页] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

                # = = = = = = = = = = = = = = =

            if not list_dict_correct_temp:
                raise

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Json_Compatibility', 'Page', 'Row'],
                                                                 ascending=[True, True, True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

总数量：10

[第1页] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/24 09:55:57] - 2. https://www.ebay.com/itm/335270737835?_ul=US&_stpos=91710&orig_cvip=true

[第1页] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/24 09:55:57] - 9. https://www.ebay.com/itm/334348192867?_ul=US&_stpos=91710&orig_cvip=true

[第1页] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/24 09:55:57] - 4. https://www.ebay.com/itm/235748987359?_ul=US&_stpos=91710&orig_cvip=true

[第1页] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/24 09:55:57] - 3. https://www.ebay.com/itm/232315264904?_ul=US&_stpos=91710&orig_cvip=true

[第1页] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/24 09:55:57] - 10. https://www.ebay.com/itm/232479816006?_ul=US&_stpos=91710&orig_cvip=true

[第1页] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/24 09:55:58] - 6. https://www.ebay.com/itm/235461387733?_ul=US&_stpos=91710&orig_cvip=true

[第1页] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/24 09:55:58] - 8. https://www.ebay.com/i

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4937.38it/s]

Done ~
